In [ ]:
import torch
import temp
import torch.nn as nn
import torchvision.models as model
from torchvision import transforms as trn
import os
import sys
from PIL import Image
from torch.autograd import Variable as V
import torchvision.models as models
from torch.nn import functional as F
import torch.optim as optim
from sklearn.svm import SVC
from torch.utils.data import DataLoader
from copy import deepcopy
from torch.autograd import Variable
from torchvision import datasets, models, transforms
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.utils import shuffle
import numpy as np

mean = [0.4856586910840433, 0.4856586910840433, 0.4856586910840433]
std = [0.14210993338737993, 0.14210993338737993, 0.14210993338737993]

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomSizedCrop(220),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'validation': transforms.Compose([
        transforms.Scale(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
}

In [ ]:
criterion = nn.CrossEntropyLoss()
model = model.cuda()
model.eval()

model = torch.nn.DataParallel(model)

In [ ]:
class BYNET(nn.Module):
    def __init__(self, num_classes=15):
        super(BYNET, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=(14,14), stride=(14,14))
        )
        self.layer_relu = nn.Sequential(
            nn.ReLU(),
            nn.Linear(512*4, 1024)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(1024, 1024, kernel_size=(7, 7), stride=(7,7))
        )
        self.layer2_relu = nn.Sequential(
            nn.ReLU(),
            nn.Linear(1024*4, 1024)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(2048, 2048, kernel_size=(4, 4), stride=(3,3))          
        )
        self.layer3_relu = nn.Sequential(
            nn.ReLU(),
            nn.Linear(2048*4, 1024)
        )
        
        self.fc1 = nn.Linear(1024*3,1024)
        self.fc2 = nn.Linear(1024, 395)
    
    def forward(self, x):
        out1 = self.layer1(x[0])
        out1 = out1.view(-1, num_flat_features(out1))
        out1 = self.layer_relu(out1) 
        
        out2 = self.layer2(x[1])
        out2 = out2.view(-1, num_flat_features(out2))
        out2 = self.layer2_relu(out2) 
        
        out3 = self.layer3(x[2])
        out3 = out3.view(-1, num_flat_features(out3))
        out3 = self.layer3_relu(out3) 
        
        out = torch.cat((out1,out2,out3),1)
        out = self.fc1(out)
        out = self.fc2(out)
        
        return out

In [ ]:
def num_flat_features(x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [ ]:
def train_model(model_ft,optimizer,scheduler,net,num_epochs):
    
    best_model_wts = deepcopy(net.state_dict())
    best_acc = 0.0
        
    for epoch in range(1,num_epochs+1):
        for phase in ['train','validation']:
            if phase == 'train':
                scheduler.step()
                print('\n=> Training Epoch #%d' %(epoch))
                net.train()
            else:
                print('\n=> Validation Epoch #%d' %(epoch))
                net.eval()

            running_loss = 0.0       
            running_corrects = 0
            tot = 0.0

            for batch_idx, (inputs, target) in enumerate(dset_loaders[phase]):
                inputs, target = Variable(inputs.cuda()), Variable(target.cuda())

                #추가된 layer 이외에 다른 layer는 학습 x
                for param in model.parameters():
                    param.requires_grad = False

                optimizer.zero_grad()


                out1 = model_ft.module.conv1.forward(inputs)
                out2 = model_ft.module.relu(model_ft.module.bn1(out1))
                out3 = model_ft.module.maxpool.forward(out2)

                layer1_feature = model_ft.module.layer1.forward(out3)
                layer2_feature = model_ft.module.layer2.forward(layer1_feature)
                layer3_feature = model_ft.module.layer3.forward(layer2_feature)
                layer4_feature = model_ft.module.layer4.forward(layer3_feature)

                layer_feature_list = list()
                layer_feature_list.append(layer2_feature)
                layer_feature_list.append(layer3_feature)
                layer_feature_list.append(layer4_feature)

                output = net.forward(layer_feature_list)    
                _,preds = torch.max(output.data,1)
                loss  = criterion(output, target)
                
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                running_loss += loss.item()
                running_corrects += preds.eq(target.data).cpu().sum()
                tot += target.size(0)

                if phase == 'train':
                    sys.stdout.write('\r')
                    sys.stdout.write('| Epoch [%2d/%2d] Iter [%3d/%3d]\t\tLoss %.4f\tAcc %.2f%%'
                            %(epoch, num_epochs, batch_idx,
                                (len(dsets[phase])//190)+1, loss.data[0], 100.*running_corrects/tot))
                    sys.stdout.flush()
                    sys.stdout.write('\r')
                if phase == 'validation':
                    print("===> Validation Epoch[{}]({}/{}): Loss: {:.4f}: Accuracy: {:.4f}".format(epoch, batch_idx, dset_sizes['validation'], loss.data[0],100.*running_corrects/tot))

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            if phase == 'validation' and epoch_acc > best_acc:
                print('\n| Validation Epoch #%d\t\t\tLoss %.4f\tAcc %.2f%%'
                    %(epoch, loss.data[0], 100.*epoch_acc))
                best_acc = epoch_acc
                best_model_wts = deepcopy(net.state_dict())

                if not os.path.isdir('checkpoint'):
                    os.mkdir('checkpoint')
                save_point = './checkpoint/'
                if not os.path.isdir(save_point):
                    os.mkdir(save_point)
                torch.save(best_model_wts, '{}.pth'.format('BYNET_best_sun397'))
        
        net_copy = deepcopy(net.state_dict())
        torch.save(net, '{}.pth'.format(epoch))
        
    print('Best validation Acc\t{:.2f}%'.format(best_acc*100))
    
    return best_model_wts

In [ ]:
net = BYNET().cuda()
optimizer = torch.optim.SGD(net.parameters(), lr=0.1,weight_decay=1e-2)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,milestones=learning_milestones,gamma=0.8)
net_ft = train_model(model, optimizer,scheduler,net,100)